In [1]:
%env THEANO_FLAGS=device=gpu1,floatX=float32

import lasagne

from lasagne.layers import *
import theano 
import theano.tensor as T
import numpy as np

from tqdm import tqdm

env: THEANO_FLAGS=device=gpu1,floatX=float32


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX 1080 (CNMeM is enabled with initial size: 22.0% of memory, cuDNN 5110)


In [2]:
with open('zbornique.txt', 'r') as t:
    text = t.read().lower()
    text = unicode(text, 'cp1251').lower()

In [3]:
garbage = [u'*', u'/', u'<', u'>', '=', ']', '[', '«', '»', '…', '	']
for sym in garbage:
    try:
        text = text.replace(sym, '')
    except:
        print 'Could not encode ', sym 

Could not encode  «
Could not encode  »
Could not encode  …


In [4]:
chars = sorted(set(text))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for i, c in enumerate(chars)}

vocab_size = len(chars)

In [5]:
X = list()
Y = list()
seq_len = 100
step = 1

for i in range(0, len(text) - seq_len - step, step):
    seq = [char2idx[char] for char in text[i:i+seq_len]]
    char = char2idx[text[i+seq_len]]
    
    X.append(seq)
    Y.append(char)
    
X = np.asarray(X, dtype='int32')
y = np.asarray(Y, dtype='int32')

Y = np.zeros([len(y), vocab_size])
Y[np.arange(len(y)), y] = 1

In [6]:
inputShape = [None, seq_len]
inputX = T.matrix('X', dtype='int32')
inputY = T.matrix('Y', dtype='int32')

In [7]:
nn = InputLayer(inputShape, inputX)
nn = EmbeddingLayer(nn, vocab_size, 1024)

nn = LSTMLayer(nn, 512)
nn = DropoutLayer(nn, 0.8)

nn = LSTMLayer(nn, 512)
nn = DropoutLayer(nn, 0.8)

nn = LSTMLayer(nn, 512, only_return_final=True)
nn = DropoutLayer(nn, 0.8)

nn = DenseLayer(nn, vocab_size, nonlinearity=T.nnet.softmax)

In [7]:
import pickle

In [8]:
with open('layers') as f:
    layers = pickle.load(f)

In [11]:
layers[0].input_var = inputX
nn = layers[-1]

In [12]:
probs = get_output(nn)
weights = get_all_params(nn, trainable=True)

In [13]:
loss = lasagne.objectives.categorical_crossentropy(probs, inputY).mean()
updates = lasagne.updates.adam(loss, weights)

In [14]:
train_fun = theano.function([inputX, inputY], loss, updates=updates, allow_input_downcast=True)
predict = theano.function([inputX], probs, allow_input_downcast=True)

In [12]:
n_epochs = 27
batch_size = 3020

for epoch in range(n_epochs):
    error = 0
    a = 0
    for i in tqdm(range(0, len(Y) - batch_size, batch_size)):
        error+=train_fun(X[i:i+batch_size], Y[i:i+batch_size])
        a+=1
    print 'epoch: ', epoch, 'mean loss: ', float(error)/float(a)


  0%|          | 0/57 [00:00<?, ?it/s]


MemoryError: Error allocating 1236992000 bytes of device memory (CNMEM_STATUS_OUT_OF_MEMORY).
Apply node that caused the error: GpuReshape{2}(GpuDimShuffle{1,0,2}.0, MakeVector{dtype='int64'}.0)
Toposort index: 244
Inputs types: [CudaNdarrayType(float32, 3D), TensorType(int64, vector)]
Inputs shapes: [(100, 3020, 1024), (2,)]
Inputs strides: [(1024, 102400, 1), (8,)]
Inputs values: ['not shown', array([302000,   1024])]
Outputs clients: [[GpuDimShuffle{1,0}(GpuReshape{2}.0), GpuDot22(GpuReshape{2}.0, GpuJoin.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [15]:
def pred2char(pred):
    return idx2char[pred.argmax()]

In [25]:
import random

In [52]:
poem = text[5000:5100]

In [53]:
 for i in range(1000):
    fun = poem[len(poem)-100:]
    x = np.asarray([[char2idx[char] for char in fun]], dtype='float32')

    char = idx2char[predict(x).argmax()]
    poem = ''.join([poem, char])

In [54]:
print poem[100:]
print len(poem)

 с проводом старик под старок свет,
на старо в всем и страстью под странной сладком послал,
и с тобой покойный по страсть послушай
и старик в свет в солданный продорститься.
не в старик постели поспетит,
и с старовой постолвеньи стран
в поменный старик в страшной свете в старинем,
в сердца поседела в соборный полет он.




с постара


с тобою последний стал,
с постойных старом престой,
и с весельем постели.









а

восливи в под порор





послания стал. 




послединея





 постонот





стопое послания покомной





ркаки мниской постал повеселье стедьва сторон по старой столой постолнек стар и с велимой с в под в постолнум полтожит в отдари на пернатьи по то страдали с страшной покойный старик м от е старик посторние покорной пристал 





послание 1 покаста скажет.




к сон стихи постесен весерье с простовом


постор с постонницей полетит,
на стали с молодой под страми стеной,
по долед
1100


In [11]:
a = 5000
assert idx2char[X[a+1][-1]] == idx2char[Y[a].argmax()]

In [14]:
import pickle

In [15]:
with open('layers', 'wb') as f:
    pickle.dump(get_all_layers(nn), f)